## Summing all the prime numbers below a given number

In [1]:
import sys
#parámetro desde la línea de comandos
number=2_500_000

if len(sys.argv) > 1: #Usa el último argumento de la línea si es un entero
    last = sys.argv[-1]
    if last.isdigit():
        number = int(last)

print(f"El valor de 'number' es: {number}")

El valor de 'number' es: 2500000


In [2]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.769779046376546
The prime sum below  2500000 is  219697708195  and the time taken is 4.61 s ± 6.73 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado a) Aplicando el paquete Numba

In [3]:
from numba import njit


# Simple code

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.6701639493306478
The prime sum below  2500000 is  219697708195  and the time taken is 184 ms ± 26.1 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado b) Usando Multiprocessing con Pool

In [4]:
from multiprocessing import Pool

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

def sum_primes_interval(args): #función worker, necesaria para ejecución en paralelo
    a, b = args
    result=0
    for i in range(a,b):
        result += if_prime(i)
    return result

def sum_primes_parallel(number, ncores): #funcion orquestadora que divide el trabajo entre los 4 núcleos
    step= number// ncores
    intervals= [ #lista de tuplas
    (0,step),
    (step, 2*step),
    (2*step, 3*step),
    (3*step, number)
]
    with Pool(processes=ncores) as p:
        partial_sums=p.map(sum_primes_interval, intervals) #cada proceso recibe una tupla y solo suma en ese rango
    return sum(partial_sums)
        

suma = 0
N = 3 # number of loops
ncores=4

start = time.time()
for i in range(N):
    suma=sum_primes_parallel(number, ncores)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_parallel(number,ncores)
suma = sum_primes_parallel(number,ncores)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.26684006055196124
The prime sum below  2500000 is  219697708195  and the time taken is 264 ms ± 80.5 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


Apartado c) Usando Numba con prange

from numba import njit, prange, set_num_threads, get_num_threads

@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

@njit(parallel=True) #creamos una función para paralelizar con prange de Numba
def sum_primes_prange(x):
    result=0
    for i in prange(x):
        result += if_prime(i)
    return result
        
suma = 0
N = 3 # number of loops

set_num_threads(4) #para usar 4 núcleos
print("Numba threads:", get_num_threads())

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


## **SALIDAS DE SLURM**

**EJECUCIÓN CON 1 NÚCLEO**

El valor de 'number' es 10**6
1. Código original de Python usando: time.time()= 2.2517224152882895 /%time.it= 2.21 s ± 2.48 ms per loop
2. Numba (@njit): time.time()= 1.5764572620391846 /%time.it= 101 ms ± 46.2 μs per loop
3. Multiprocessing con Pool: time.time()=0.2712765534718831 /%time.it= 256 ms ± 714 μs per loop
4. Numba con prange: time.time()= 0.37956730524698895/ %time.it= 37.8 ms ± 151 μs per loop

El valor de 'number' es: 10**7
1. Código original de Python usando: time.time()= 59.002935806910195 /%time.it= 57.1 s ± 3.2 ms per loop
2. Numba (@njit): time.time()= 5.132996877034505 /%time.it= 2.54 s ± 33.9 μs per loop
3. Multiprocessing con Pool: time.time()= 1.572932243347168 /%time.it= 1.56 s ± 1.75 ms per loop
4. Numba con prange: time.time()= 1.273139476776123 /%time.it= 957 ms ± 109 μs per loop

**EJECUCIÓN CON 2 NÚCLEOS**

El valor de 'number' es 10**6
1. Código original de Python usando: time.time()= 2.2760423024495444 /%time.it= 2.24 s ± 98.7 μs per loop
2. Numba (@njit): time.time()= 1.572581132253011 /%time.it= 101 ms ± 51 μs per loop
3. Multiprocessing con Pool: time.time()= 0.2689970334370931 /%time.it= 250 ms ± 2.68 ms per loop
4. Numba con prange: time.time()= 0.3556505839029948/ %time.it= 37.7 ms ± 55 μs per loop

El valor de 'number' es: 10**7
1. Código original de Python usando: time.time()= 59.65240836143494 /%time.it= 57.5 s ± 134 μs per loop
2. Numba (@njit): time.time()= 4.802578528722127 /%time.it= 2.54 s ± 72.3 μs per loop
3. Multiprocessing con Pool: time.time()= 1.5631403923034668 /%time.it= 1.55 s ± 5.82 ms per loop
4. Numba con prange: time.time()= 1.2532780170440674 /%time.it= 957 ms ± 128 μs per loop


**EJECUCIÓN CON 4 NÚCLEOS**

El valor de 'number' es 10**6
1. Código original de Python usando: time.time()= 2.294480482737223 /%time.it= 2.24 s ± 1.01 ms per loop
2. Numba (@njit): time.time()= 0.5934070746103922 /%time.it= 101 ms ± 836 ns per loop
3. Multiprocessing con Pool: time.time()= 0.25014781951904297 /%time.it= 244 ms ± 212 μs per loop
4. Numba con prange: time.time()= 0.34086330731709796/ %time.it= 37.6 ms ± 2.65 μs per loop

El valor de 'number' es: 10**7
1. Código original de Python usando: time.time()= 59.325949589411415 /%time.it= 57.3 s ± 6.4 ms per loop
2. Numba (@njit): time.time()= 4.819015264511108 /%time.it= 2.54 s ± 440 μs per loop
3. Multiprocessing con Pool: time.time()= 1.5670993328094482 /%time.it= 1.56 s ± 7 ms per loop
4. Numba con prange: time.time()= 1.2536199887593586 /%time.it= 957 ms ± 184 μs per loop

**EJECUCIÓN CON 8 NÚCLEOS**

El valor de 'number' es 10**6
1. Código original de Python usando: time.time()= 2.252448320388794 /%time.it= 2.18 s ± 596 μs per loop
2. Numba (@njit): time.time()= 0.5228908856709799 /%time.it= 101 ms ± 77.6 μs per loop
3. Multiprocessing con Pool: time.time()= 0.24902812639872232 /%time.it= 245 ms ± 8.89 μs per loop
4. Numba con prange: time.time()= 0.33800578117370605/ %time.it= 37.6 ms ± 2.95 μs per loop
   
El valor de 'number' es: 10**7
1. Código original de Python usando: time.time()= 59.13050874074300 /%time.it= 57.3 s ± 110 ms per loop
2. Numba (@njit): time.time()= 4.838505268096924 /%time.it= 2.54 s ± 411 μs per loop
3. Multiprocessing con Pool: time.time()= 1.56532088915507 /%time.it= 1.55 s ± 4.89 ms per loop
4. Numba con prange: time.time()= 1.2792261441548665 /%time.it= 957 ms ± 23.3 μs per loop

## **CONCLUSIÓN DE LOS 4 ENFOQUES ESTUDIADOS:**

Aunque los tiempos medidos dentro de Python permiten comparar el rendimiento relativo de los distintos métodos, los tiempos obtenidos a través de SLURM reflejan el coste real de ejecución en el clúster siendo por tanto los más representativos en un entorno HPC.
Los tiempos obtenidos en cada caso muestran las diferencias entre cada enfoque usado. El código original en Python presenta el tiempo de ejecución más elevado al ser el código primitivo y no optimizado. Aplicando el paquete Numba (@njit) sobre el código anterior, el tiempo se reduce considerablemente al compilar las funciones a código maquina, sin embargo, sigue siendo código secuencial, por lo que se usa un único núcleo. Las mejoras más notables se observan al paralelizar el programa usando multiprocessing y Numba con prange. En el primero de estos casos el tiempo se ve reducido al dividir el trabajo entre varios procesos, sin embargo, el tiempo es mayor que al usar Numba con prange, esto es debido a que existe un coste extra de la gestión y creación de los procesos. Por último, usando prange de Numba, el tiempo de ejecución se ha visto reducido considerablemente debido a que ahora se paraleliza usando 4 hilos, pero no se crea ningún proceso nuevo, por lo que se evita el coste que aumentaba el tiempo cuando usabamos multiprocessing.

En cuanto a la comparación de los tiempos obtenidos en función del número de núcleos, se observa sobre todo que el paralelismo es más beneficioso cuando se trabaja con problemas más grandes, en  nuestro caso 10**7. Observandio las salidas de Python y SLURM se puede concluir que los tiempo de SLURM muestran el coste real de ejecución en el clúster mientras que los de Python sirven más para comparar con mayor precisión cuánto de eficaz es cada enfoque usado.